### 추가 설치 필요

In [59]:
# 한 번만 설치하면 되는 것들은 설치 후 주석 처리. 필요 시 주석 풀고 설치
# !pip install requests
# !pip install aiohttp
# !pip install geopy
# !pip install geopandas
# !pip install shapely
# !pip install pyproj

In [60]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from shapely.geometry import Point
import geopandas as gpd
from geopandas.tools import sjoin
import pyproj
from tqdm import tqdm

### 아파트와 지하철 역의 위치 데이터를 사용하여 밀도를 계산하는 함수

In [61]:
# bus-station density, subway-station density

#1. 아파트 위도와 경도 정보를 UTM-K 좌표계로 변환하고, 이를 기반으로 지오메트리 컬럼을 생성합니다.
def preprocess_data(df, x_col, y_col):
    wgs84 = pyproj.CRS('EPSG:4326')  # WGS84 좌표계 (위도, 경도)
    utm_k = pyproj.CRS('EPSG:5179')  # UTM-K 좌표계 (한국)
    project = pyproj.Transformer.from_crs(wgs84, utm_k, always_xy=True).transform
    
    df['geometry'] = df.apply(lambda row: Point(project(row[x_col], row[y_col])), axis=1)
    return gpd.GeoDataFrame(df, geometry='geometry', crs=utm_k)

# 2. 위치 데이터를 사용하여 주어진 반경 내의 밀도를 계산합니다.
def calculate_density(gdf_real_estate, gdf_locations, radius):
    gdf_real_estate['buffer'] = gdf_real_estate.geometry.buffer(radius)
    
    joined = sjoin(gdf_locations, gdf_real_estate.set_geometry('buffer'), how='inner', predicate='within')
    
    density = joined.groupby(joined.index_right).size().rename('density')
    
    return gdf_real_estate.join(density, how='left').fillna(0)['density']


# 큰 데이터프레임을 작은 청크로 나누어 처리하여 메모리 사용을 최적화
def process_in_chunks(df, locations_gdf, radius, chunk_size=10000):
    results = []
    for i in tqdm(range(0, len(df), chunk_size)):
        chunk = df.iloc[i:i+chunk_size]
        chunk_gdf = preprocess_data(chunk, '좌표X', '좌표Y')
        density = calculate_density(chunk_gdf, locations_gdf, radius)
        results.append(density)
    return pd.concat(results)

### 데이터 로드

In [62]:
path = '../trainTestLatLng.csv'
bus_path  = '../bus_feature.csv'
subway_path  = '../subway_feature.csv'

# train + test + 결측치 채운값
concat = pd.read_csv(path, index_col=0)
#버스
dt_bus = pd.read_csv(bus_path)
#지하철
dt_subway = pd.read_csv(subway_path)

/tmp/ipykernel_495348/854141158.py:6: DtypeWarning: Columns (14,15,17,18,37) have mixed types. Specify dtype option on import or set low_memory=False.
  concat = pd.read_csv(path, index_col=0)


In [63]:
display(len(concat))
display(concat.head(1))
display(dt_bus.head(1))
display(dt_subway.head(1))

1128094

,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000.0,0


,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로


,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877


In [64]:
# 버스 정류장과 지하철역 데이터 전처리
bus_gdf = preprocess_data(dt_bus, 'X좌표', 'Y좌표')  # 버스 정류장 좌표계 변환
bus_gdf.head(2)

,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입,geometry
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로,POINT (954764.39 1952394.348)
1,100000002,1002,창경궁.서울대학교병원,126.996566,37.579183,중앙차로,POINT (955548.315 1953430.255)


In [65]:
# 지하철 데이터는 위도, 경도 데이터를 X, Y로 변환
subway_gdf = preprocess_data(dt_subway, '경도', '위도')  # 지하철 데이터도 동일하게 좌표 변환
subway_gdf.head(2)

,역사_ID,역사명,호선,위도,경도,geometry
0,9996,미사,5호선,37.560927,127.193877,POINT (972963.732 1951329.788)
1,9995,강일,5호선,37.557490,127.175930,POINT (971377.365 1950953.787)


In [66]:
# 반경 설정
bus_radius = 500  # 미터 단위
subway_radius = 4000  # 1km를 미터 단위로 표현

# 청크 크기 설정
chunk_size = 10000

In [67]:
print("버스 정류장 밀도 계산 중 (500m 반경)...")
concat['bus_stop_density_500m'] = process_in_chunks(concat, bus_gdf, bus_radius, chunk_size)

print("지하철역 밀도 계산 중 (1km 반경)...")
concat['subway_station_density_1km'] = process_in_chunks(concat, subway_gdf, subway_radius, chunk_size)

# 결과 출력
print(concat[['좌표X', '좌표Y', 'bus_stop_density_500m', 'subway_station_density_1km']].head())

버스 정류장 밀도 계산 중 (500m 반경)...


  0%|          | 0/113 [00:00<?, ?it/s]/tmp/ipykernel_495348/1140538833.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df.apply(lambda row: Point(project(row[x_col], row[y_col])), axis=1)
  1%|          | 1/113 [00:00<00:57,  1.94it/s]/tmp/ipykernel_495348/1140538833.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df.apply(lambda row: Point(project(row[x_col], row[y_col])), axis=1)
  2%|▏         | 2/113 [00:01<00:56,  1.95it/s]/tmp/ipykernel_495348/1140538833.py:9

지하철역 밀도 계산 중 (1km 반경)...


  0%|          | 0/113 [00:00<?, ?it/s]/tmp/ipykernel_495348/1140538833.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df.apply(lambda row: Point(project(row[x_col], row[y_col])), axis=1)
  1%|          | 1/113 [00:00<01:22,  1.36it/s]/tmp/ipykernel_495348/1140538833.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df.apply(lambda row: Point(project(row[x_col], row[y_col])), axis=1)
  2%|▏         | 2/113 [00:01<01:20,  1.37it/s]/tmp/ipykernel_495348/1140538833.py:9

         좌표X        좌표Y  bus_stop_density_500m  subway_station_density_1km
0  127.05721  37.476763                     13                          25
1  127.05721  37.476763                     13                          25
2  127.05721  37.476763                     13                          25
3  127.05721  37.476763                     13                          25
4  127.05721  37.476763                     13                          25


In [68]:
# 가중치를 적용:
bus_weight = 0.5  # 예: 버스 밀도에 가중치 0.5
subway_weight = 0.8  # 예: 지하철 밀도에 가중치 0.7

concat['대중교통가중치'] = (concat['bus_stop_density_500m'] * bus_weight + 
                                       concat['subway_station_density_1km'] * subway_weight)
                                       
concat= concat.drop(columns=['bus_stop_density_500m','subway_station_density_1km'])

In [69]:
concat.shape
print(concat.columns)


Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target', 'is_test', '대중교통가중치'],
      dtype='object')


In [70]:
concat.head(1)


,시군구,번지,본번,부번,아파트명,전용면적,계약년월,계약일,층,건축년도,...,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target,is_test,대중교통가중치
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000.0,0,26.5


In [71]:
concat.to_csv("../trainTestLatLngBusSubway.csv")